# WiFi Connection

In [ ]:
import network
import time

print("Connecting to WiFi", end="")
sta_if = network.WLAN(network.STA_IF)
sta_if.active(True)
sta_if.connect('Wokwi-GUEST', '')
while not sta_if.isconnected():
  print(".", end="")
  time.sleep(0.1)
print(" Connected!")
print('network config:', sta_if.ifconfig())

# HTTP Client

In [ ]:
import urequests as requests

import json

def get_prediction(data = {'some-header': '20'}):
  url = 'https://summerschool.free.beeceptor.com'
  r = requests.post(url, data=json.dumps(data))
  print(r)
  response = r.text
  print(response)
  return response

get_prediction()

# HTTP server

In [ ]:
import socket
from machine import Pin

led = Pin(2, Pin.OUT)

def web_page():
  if led.value() == 1:
    gpio_state="ON"
  else:
    gpio_state="OFF"

  html = """<html><head> <title>ESP Web Server</title> <meta name="viewport" content="width=device-width, initial-scale=1">
  <link rel="icon" href="data:,"> <style>html{font-family: Helvetica; display:inline-block; margin: 0px auto; text-align: center;}
  h1{color: #0F3376; padding: 2vh;}p{font-size: 1.5rem;}.button{display: inline-block; background-color: #e7bd3b; border: none;
  border-radius: 4px; color: white; padding: 16px 40px; text-decoration: none; font-size: 30px; margin: 2px; cursor: pointer;}
  .button2{background-color: #4286f4;}</style></head><body> <h1>ESP Web Server</h1>
  <p>GPIO state: <strong>""" + gpio_state + """</strong></p><p><a href="/?led=on"><button class="button">ON</button></a></p>
  <p><a href="/?led=off"><button class="button button2">OFF</button></a></p></body></html>"""
  return html

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind(('0.0.0.0', 80))
s.listen(5)

while True:
  conn, addr = s.accept()
  print('Got a connection from %s' % str(addr))
  request = conn.recv(1024)
  request = str(request)
  print('Content = %s' % request)
  led_on = request.find('/?led=on')
  led_off = request.find('/?led=off')
  if led_on == 6:
    print('LED ON')
    led.value(1)
  if led_off == 6:
    print('LED OFF')
    led.value(0)
  response = web_page()
  conn.send('HTTP/1.1 200 OK\n')
  conn.send('Content-Type: text/html\n')
  conn.send('Connection: close\n\n')
  conn.sendall(response)
  conn.close()

# MQTT - Publish data
- Online MQTT client: https://www.hivemq.com/demos/websocket-client/

In [ ]:
# Connect to a broker
# main.py
from umqtt.simple import MQTTClient

CLIENT_NAME = 'abcd'
BROKER_ADDR = 'broker.mqttdashboard.com'
mqttc = MQTTClient(CLIENT_NAME, BROKER_ADDR, keepalive=60)
mqttc.connect()

# Publish data
TOPIC = 'atul_test_1234'
MSG = 'Hi from ESP32'
while True:
	mqttc.publish( TOPIC, MSG.encode())
	time.sleep(0.5)

# MQTT - Subscribe data

In [ ]:
from umqtt.simple import MQTTClient

# Define callback function for handling received MQTT messages
def sub_cb(topic, msg):
    print((topic, msg))

# Define MQTT broker parameters
mqtt_broker = 'broker.mqttdashboard.com'
mqtt_topic = 'mqtt_topic_to_subscribe'

# Create an MQTT client instance
client = MQTTClient(client_id='esp32', server=mqtt_broker)

# Connect to the MQTT broker
client.connect()

# Subscribe to the MQTT topic
client.set_callback(sub_cb)
client.subscribe(mqtt_topic)

# Main loop
while True:
    try:
        # Wait for MQTT messages
        client.check_msg()
    except OSError as e:
        print("Failed to read MQTT message: %s" % e)

# Blink LED

In [ ]:
import network
import time
from machine import Pin

print("Connecting to WiFi", end="")
sta_if = network.WLAN(network.STA_IF)
sta_if.active(True)
sta_if.connect('Wokwi-GUEST', '')
while not sta_if.isconnected():
  print(".", end="")
  time.sleep(0.1)
print(" Connected!")
print('network config:', sta_if.ifconfig())

from umqtt.simple import MQTTClient

led = Pin(2, Pin.OUT)
# Define callback function for handling received MQTT messages
def sub_cb(topic, msg):
  print((topic, msg))
  if int(msg.decode()) == 1:
    led.value(1)
  else:
    led.value(0)


# Define MQTT broker parameters
mqtt_broker = 'broker.mqttdashboard.com'
mqtt_topic = '123atul432'

# Create an MQTT client instance
client = MQTTClient(client_id='esp32', server=mqtt_broker)

# Connect to the MQTT broker
client.connect()

# Subscribe to the MQTT topic
client.set_callback(sub_cb)
client.subscribe(mqtt_topic)

# Main loop
while True:
    try:
        # Wait for MQTT messages
        client.check_msg()
    except:
        pass